In [ ]:
# Convertir código de jupyter notebook + los resultados a html, pdf y word

# 1) Tienes que instalar "Quarto".
# 2) Para pdf tienes que isntalar "MinTeX".
# 3) Tu archivo Jupyter Notebook debe estar guardada.

# Estos son los comandos:

# quarto render C:/nombre_archivo_jupyternotebook.ipynb
# quarto render C:/nombre_archivo_jupyternotebook.ipynb -- to pdf
# quarto render C:/nombre_archivo_jupyternotebook.ipynb --to docx

# quarto render C:\Users\usuario\Documents\GitHub\Aprendizaje-automatico-con-python\Bloque 1 - Introducción al aprendizaje automático\PEC\M2.891_20251_PEC1-UbedaQuesadaJulio.ipynb

# NOTA:
# Este comando debes escribirlo en la terminal.
# Debes quitar el "#" al comando,
# Debes escribir la ruta de tu archivo de código a convertir.